# Построение нейронной сети на корпусе текстов RuSentiment для задачи анализа тональности

In [0]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [0]:
!pip install pymorphy2
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re




from scipy.sparse import hstack

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_validate

from sklearn.preprocessing import LabelEncoder


from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report, f1_score

from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import pymorphy2
import string

import re
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors, doc2vec, FastText
from sklearn.utils import shuffle

from pickle import dump



from pymystem3.mystem import Mystem
myst = Mystem()

     |████████████████████████████████| 51kB 2.0MB/s 
     |████████████████████████████████| 7.1MB 8.0MB/s 


Using TensorFlow backend.
Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [0]:
from google.colab import drive 
drive.mount('/content/gdrive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import sys 
sys.path.append('/content/gdrive/уMy Drive/Colab Notebooks') 


In [0]:
data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/rusent.csv')
data.head()

,label,text
0,negative,"А попа подозревала давно,что ты с кавказа..пер..."
1,speech,З прошедшим Днем Ангела))))))))
2,skip,Два дня до отлёта с острова!!!!!!!
3,negative,"Блин, почему эта жизнь столь не справедлива (((("
4,skip,где еще встречать свой день рождения как не на...


In [0]:

data = data[(data.label!='skip') & (data.label != 'speech') & (data.label != 'neutral')]


In [0]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = set(stopwords.words('russian'))
stopwords = set(stopwords.words('russian'))
print(stop)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
{'всех', 'без', 'во', 'этого', 'сам', 'на', 'вам', 'они', 'три', 'тем', 'всего', 'чем', 'разве', 'от', 'меня', 'да', 'уже', 'у', 'вот', 'сейчас', 'до', 'про', 'вас', 'себе', 'хоть', 'и', 'мы', 'наконец', 'ж', 'никогда', 'по', 'них', 'впрочем', 'уж', 'ли', 'были', 'было', 'она', 'когда', 'где', 'ведь', 'какая', 'быть', 'моя', 'им', 'же', 'ни', 'раз', 'при', 'об', 'для', 'будто', 'с', 'все', 'вдруг', 'том', 'чтоб', 'куда', 'ей', 'не', 'этом', 'тогда', 'этот', 'под', 'один', 'будет', 'лучше', 'еще', 'другой', 'тот', 'или', 'ней', 'конечно', 'мой', 'себя', 'тоже', 'можно', 'кто', 'перед', 'я', 'была', 'после', 'чуть', 'если', 'почти', 'надо', 'эти', 'а', 'только', 'между', 'ее', 'два', 'то', 'со', 'чтобы', 'к', 'хорошо', 'в', 'там', 'нас', 'тебя', 'ему', 'потому', 'свою', 'такой', 'что', 'есть', 'нее', 'много', 'чего', 'опять', 'из', 'он', 'зачем', 'через', 'иногда', 'того', 'их',

In [0]:
data.head()

,label,text
0,negative,"А попа подозревала давно,что ты с кавказа..пер..."
3,negative,"Блин, почему эта жизнь столь не справедлива (((("
6,positive,патамушта я крутая баба! ееее!!
8,positive,урря!я дождался этой овцыы)
11,positive,Наша гордость !


# Предобработка текстовых данных

In [0]:
#stopwords = set(stopwords.words('russian'))
morph = pymorphy2.MorphAnalyzer()

def text_processing(data):
    t_arr = TweetTokenizer().tokenize(data)
    cleaned_data = []
    for word in t_arr:
        if word.lower() not in stopwords:
            if word.isalnum() and word != 'RT':
                lemma = morph.parse(word.lower())[0].normal_form
                cleaned_data.append(lemma)
        else:
            pass
        fin_str = ' '.join(cleaned_data)
    return fin_str

def hands_re(text):
    punctuation = string.punctuation + '\u2014\u2013\u2012\u2010\u2212' + '«»‹›‘’“”„`'
    word_tokenize = re.compile(r"([^\w_\u2019\u2010\u002F-]|[+])")
    tokenized_words = []
    for token in word_tokenize.split(text):
        if token and not token.isspace() and not token in punctuation: # если слово не попадает в те которые мы исключаем
            tokenized_words.append(token.lower())        # добавляем слово в список
    return tokenized_words                     # возвращаем список слов

In [0]:
data['label'].values

array(['negative', 'negative', 'positive', ..., 'positive', 'positive',
       'positive'], dtype=object)

In [0]:
clean_data = []

for text in data['text']:
    new_text = text_processing(text)
    clean_data.append(new_text)

In [0]:
X = clean_data
y = data['label'].values

In [0]:
X

['поп подозревать давно кавказ перестать общаться ты',
 'блин почему этот жизнь столь справедливый',
 'патамушт крутой баба еее',
 'уррить дождаться овцы',
 'наш гордость',
 'поход один поздравить 1 апрель это ржд 40 мина ждать электричка',
 'тесак арестовать',
 'мой маленький счастие любить очень сильно',
 'ролик прекрасно абсолютно весь',
 'гопник деревня разбить бутылка монета слабо',
 'сказочный мир',
 'ыыы снова пополнить счёт',
 'хах витаминчик',
 'youtube ввести нововведение видеохостинг отображать просмотр собранный ролик сторонний сайт',
 'финал ббешечка красиво стильно',
 '4 год помнить',
 'соскучиться ты дружище',
 'быть жить вечно',
 'завязывать тотализатор пока деньга просадить',
 'хомячуля ты',
 'удачный лето',
 'кажется нужный директор свежесть',
 'пляж традиция',
 'поссать один снег dя себя довольный d пхахахаххах',
 'дорогой товарищ просмотр дать кинолента весь ещё быть хотеть мясцо колбаса прочий дрянь жаль жалко бедный неразумный животное проклятый мир тупой мясорубк

In [0]:
classEncoder = LabelEncoder() # Создаем объект класса
label_y = classEncoder.fit_transform(y) 

In [0]:
len(label_y)

9764

In [0]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
labelled_y = label_y.reshape((9764, 1))
encoder.fit_transform(labelled_y)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [0]:
length_of_sentences_list = np.array([len(text) for text in X])

print("\tМаксимальное число слов в тексте: {}".format(np.max(length_of_sentences_list)))
print("\tМедиана числа слов в тексте: {}".format(np.median(length_of_sentences_list)))
print("\tМатематическое ожидание числа слов в тексте: {}".format(np.mean(length_of_sentences_list)))

	Максимальное число слов в тексте: 659
	Медиана числа слов в тексте: 32.0
	Математическое ожидание числа слов в тексте: 57.87617779598525


In [0]:
label_y

array([0, 0, 1, ..., 1, 1, 1])

In [0]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

In [0]:
'''# load the whole embedding into memory
embeddings_index = dict()
f = open('model.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))'''

Loaded 249566 word vectors.


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, label_y, test_size = 0.2, random_state = 42)

In [0]:
t = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n')
t.fit_on_texts(X_train)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_X_train = t.texts_to_sequences(X_train)
encoded_X_test = t.texts_to_sequences(X_test)
print(encoded_X_train)

[[1372, 174, 3747, 758, 1573, 8, 94, 1574], [3748, 1834, 5768, 3749], [1575, 27], [25, 821, 16], [5769, 2236], [266, 3750, 21, 3751, 1214], [1576, 438, 2784, 3752, 5770, 5771, 195, 75, 5772, 274], [230, 1090, 12, 1577, 591, 35, 759], [699], [3, 1091, 2237, 1215, 45, 3, 62, 185, 1092, 592, 3, 822, 85, 14, 1578, 336, 143, 5773, 22, 1216, 90], [1835, 1836, 2238, 73, 3753, 7, 2239, 5774, 1217, 1093, 3754, 2240, 148, 1579, 3, 49, 38, 3755], [1837, 5775, 275, 95, 4, 1373, 50, 60, 1580, 6, 11, 16, 238], [51, 640, 12, 2785, 5776, 1374, 14, 163, 1838], [27, 18, 22, 3756], [1218, 310, 65], [231, 2786], [170, 1094, 58], [823, 3757], [15, 5777, 51, 61, 4, 3758, 31, 213, 5778], [7, 5779], [2241], [6, 1219, 904, 496, 5780, 5781], [23, 35, 202, 19, 65, 641, 997, 5782, 174, 2787], [70, 300, 255, 412, 128, 118, 439, 2], [1375, 1376], [329, 3759, 128, 5783, 2788, 17, 1839, 1220, 15, 49, 2242, 8, 129, 642], [5784, 5785, 5786, 33, 2789, 2243, 5787, 369, 21, 5788, 440, 5789, 38, 5790], [2244], [186, 352, 1

In [0]:
max_length = 60
padded_X_train = pad_sequences(encoded_X_train, maxlen=max_length, padding='post')
padded_X_test = pad_sequences(encoded_X_test, maxlen=max_length, padding='post')
print(padded_X_train)

[[ 1372   174  3747 ...     0     0     0]
 [ 3748  1834  5768 ...     0     0     0]
 [ 1575    27     0 ...     0     0     0]
 ...
 [ 2122    61  2936 ...     0     0     0]
 [ 3520 14774  2139 ...     0     0     0]
 [14778  5753 14779 ...     0     0     0]]


In [0]:
padded_X_test

array([[    6,   349, 11933, ...,     0,     0,     0],
       [    8,   226,   271, ...,     0,     0,     0],
       [ 2432,     0,     0, ...,     0,     0,     0],
       ...,
       [  107,     0,     0, ...,     0,     0,     0],
       [ 3507,   545,    24, ...,     0,     0,     0],
       [  161,  3005,  1695, ...,     0,     0,     0]], dtype=int32)

In [0]:
'''model = Sequential()
e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=200, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())'''

"model = Sequential()\ne = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=200, trainable=False)\nmodel.add(e)\nmodel.add(Flatten())\nmodel.add(Dense(1, activation='sigmoid'))\n# compile the model\nmodel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])\n# summarize the model\nprint(model.summary())"

In [0]:
maxlen = 100
embedding_dim = 50
    
model = Sequential()
model.add(Embedding(vocab_size, 50, #weights=[embedding_matrix], 
                    input_length=36, #trainable=True
                   ))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 36, 50)            739150    
_________________________________________________________________
flatten_2 (Flatten)          (None, 1800)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                57632     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 796,815
Trainable params: 796,815
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
padded_X_train.shape

(7811, 60)

In [0]:
padded_X_test.shape

(1953, 60)

In [0]:
model.fit(padded_X_train, y_train, epochs=50, verbose=1,  validation_data=(padded_X_test, y_test))


Instructions for updating:
Use tf.cast instead.
Train on 7811 samples, validate on 1953 samples
Epoch 1/50
7811/7811 [==============================] - 4s 539us/step - loss: 0.5957 - acc: 0.6760 - val_loss: 0.4811 - val_acc: 0.7558
Epoch 2/50
7811/7811 [==============================] - 3s 429us/step - loss: 0.2755 - acc: 0.8921 - val_loss: 0.4996 - val_acc: 0.7839
Epoch 3/50
7811/7811 [==============================] - 3s 434us/step - loss: 0.0862 - acc: 0.9732 - val_loss: 0.5504 - val_acc: 0.7870
Epoch 4/50
7811/7811 [==============================] - 3s 424us/step - loss: 0.0376 - acc: 0.9892 - val_loss: 0.6150 - val_acc: 0.7849
Epoch 5/50
7811/7811 [==============================] - 3s 431us/step - loss: 0.0254 - acc: 0.9922 - val_loss: 0.6720 - val_acc: 0.7896
Epoch 6/50
7811/7811 [==============================] - 4s 462us/step - loss: 0.0201 - acc: 0.9928 - val_loss: 0.7356 - val_acc: 0.7849
Epoch 7/50
7811/7811 [==============================] - 3s 445us/step - loss: 0.0186 - a

In [0]:
model.fit(padded_X_train, y_train, epochs=50, verbose=1,  
          validation_data=(padded_X_test, y_test))

# evaluate the model
loss, accuracy = model.evaluate(padded_X_train, y_train, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Train on 5543 samples, validate on 1386 samples
Epoch 1/50
5543/5543 [==============================] - 2s 299us/step - loss: 0.0080 - acc: 0.9971 - val_loss: 1.3999 - val_acc: 0.7410
Epoch 2/50
5543/5543 [==============================] - 2s 296us/step - loss: 0.0079 - acc: 0.9966 - val_loss: 1.4306 - val_acc: 0.7359
Epoch 3/50
5543/5543 [==============================] - 2s 295us/step - loss: 0.0079 - acc: 0.9960 - val_loss: 1.4408 - val_acc: 0.7388
Epoch 4/50
5543/5543 [==============================] - 2s 289us/step - loss: 0.0081 - acc: 0.9966 - val_loss: 1.4349 - val_acc: 0.7410
Epoch 5/50
5543/5543 [==============================] - 2s 287us/step - loss: 0.0081 - acc: 0.9960 - val_loss: 1.4315 - val_acc: 0.7403
Epoch 6/50
5543/5543 [==============================] - 2s 296us/step - loss: 0.0078 - acc: 0.9971 - val_loss: 1.4493 - val_acc: 0.7417
Epoch 7/50
5543/5543 [==============================] - 2s 294us/step - loss: 0.0080 - acc: 0.9971 - val_loss: 1.4674 - val_acc: 0.7381


In [0]:
model.predict(X_test, y_test)

/usr/local/lib/python3.6/dist-packages/keras/engine/training_utils.py:369: RuntimeWarning: divide by zero encountered in floor_divide
  num_batches = (size + batch_size - 1) // batch_size  # round up


TypeError: ignored

In [0]:
!pip install eli5

     |████████████████████████████████| 102kB 2.6MB/s 


In [0]:
#3CVectorizer = CountVectorizer()
#X_Count = CVectorizer.fit_transform(X)



from sklearn.base import BaseEstimator, TransformerMixin
from keras.models import Model, Input
from keras.layers import Dense, LSTM, Dropout, Embedding, SpatialDropout1D, Bidirectional, concatenate
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from eli5.lime import TextExplainer
import re
import numpy as np



class KerasTextClassifier(BaseEstimator, TransformerMixin):
    '''Wrapper class for keras text classification models that takes raw text as input.'''
    
    def __init__(self, max_words=200, input_length=60, emb_dim=50, n_classes=2, epochs=25, batch_size=32):
        self.max_words = max_words
        self.input_length = input_length
        self.emb_dim = emb_dim
        self.n_classes = n_classes
        self.epochs = epochs
        self.bs = batch_size
        self.model = self._get_model()
        self.tokenizer = Tokenizer(lower=True)
    
    '''def _get_model(self):
        input_text = Input((self.input_length,))
        model = Sequential()
        model.add(Embedding(vocab_size, 50, #weights=[embedding_matrix], 
                    input_length=60, #trainable=True
                   ))
        model.add(Flatten())
        model.add(Dense(32, activation='relu'))
        model.add(Dense(1, activation = 'sigmoid'))
        model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
        return model '''

    def _get_model(self):
        input_text = Input((self.input_length,))
        text_embedding = Embedding(input_dim=self.max_words + 2, output_dim=self.emb_dim,
                                   input_length=self.input_length, mask_zero=False)(input_text)
        text_embedding = SpatialDropout1D(0.5)(text_embedding)
        bilstm = Bidirectional(LSTM(units=16, return_sequences=True, recurrent_dropout=0.5))(text_embedding)
        x = concatenate([GlobalAveragePooling1D()(bilstm), GlobalMaxPooling1D()(bilstm)])
        x = Dropout(0.7)(x)
        x = Dense(512, activation="relu")(x)
        x = Dropout(0.6)(x)
        x = Dense(512, activation="relu")(x)
        x = Dropout(0.5)(x)
        out = Dense(units=self.n_classes, activation="softmax")(x)
        model = Model(input_text, out)
        model.compile(optimizer="adam",
                      loss="sparse_categorical_crossentropy",
                      metrics=["accuracy", "mae"])
        model.summary()
        return model 
    
    def _get_sequences(self, texts):
        seqs = self.tokenizer.texts_to_sequences(texts)
        return pad_sequences(seqs, maxlen=self.input_length, value=0)
    
    def _preprocess(self, texts):
        return [re.sub(r"\d", "DIGIT", x) for x in texts]
    
    def fit(self, X, y):
        '''
        Fit the vocabulary and the model.
        
        :params:
        X: list of texts.
        y: labels.
        '''
        
        self.tokenizer.fit_on_texts(self._preprocess(X))
        self.tokenizer.word_index = {e: i for e,i in self.tokenizer.word_index.items() if i <= self.max_words}
        self.tokenizer.word_index[self.tokenizer.oov_token] = self.max_words + 1
        seqs = self._get_sequences(self._preprocess(X))
        self.model.fit(seqs, y, batch_size=self.bs, epochs=self.epochs, validation_split=0.1)
    
    def predict_proba(self, X, y=None):
        seqs = self._get_sequences(self._preprocess(X))
        return self.model.predict(seqs)
    
    def predict(self, X, y=None):
        return np.argmax(self.predict_proba(X), axis=1)
    
    def score(self, X, y):
        y_pred = self.predict(X)
        precision = sklearn.metrics.precision_score(y, y_pred)
        recall = sklearn.metrics.recall_score(y, y_pred)
        
        return sklearn.metrics.classification_report(y, y_pred)
      
    def score_accuracy(self, X, y):
        y_pred = self.predict(X)
        return accuracy_score(y, y_pred)
     
  

In [0]:
text_model = KerasTextClassifier(epochs=30, max_words=200, input_length=60)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 60)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 60, 50)       10100       input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 60, 50)       0           embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidi

In [0]:
text_model.fit(X_train, y_train)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 7029 samples, validate on 782 samples
Epoch 1/30
7029/7029 [==============================] - 20s 3ms/step - loss: 0.6639 - acc: 0.6246 - mean_absolute_error: 0.5000 - val_loss: 0.6543 - val_acc: 0.6355 - val_mean_absolute_error: 0.5000
Epoch 2/30
7029/7029 [==============================] - 18s 3ms/step - loss: 0.6362 - acc: 0.6475 - mean_absolute_error: 0.5000 - val_loss: 0.5754 - val_acc: 0.7110 - val_mean_absolute_error: 0.5000
Epoch 3/30
7029/7029 [==============================] - 18s 3ms/step - loss: 0.5981 - acc: 0.6840 - mean_absolute_error: 0.5000 - val_loss: 0.5598 - val_acc: 0.7161 - val_mean_absolute_error: 0.5000
Epoch 4/30
7029/7029 [==============================] - 18s 3ms/step - loss: 0.5908 - acc: 0.6927 - mean_absolute_error: 0.5000 - val_loss: 0.5553 - val_acc: 0.7084 - val_mean_absolute_error: 0.5000
Epoch 5/30
7029/7029 [=========

In [0]:
import sklearn
classification = text_model.score(X_test, y_test)

In [0]:
classification

'              precision    recall  f1-score   support\n\n           0       0.67      0.41      0.51       746\n           1       0.71      0.88      0.78      1207\n\n   micro avg       0.70      0.70      0.70      1953\n   macro avg       0.69      0.64      0.65      1953\nweighted avg       0.69      0.70      0.68      1953\n'

In [0]:

input_dim = X_train.shape[1]  # Number of features

model = Sequential()

model.add(Dense(10, input_dim=input_dim, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


AttributeError: 'list' object has no attribute 'shape'

In [0]:
model.compile(loss='binary_crossentropy', 
            optimizer='adam', 
            metrics=['accuracy'])
model.summary()

In [0]:
history = model.fit(X_train, y_train,
                     epochs=100,
                     verbose=False,
                     validation_data=(X_test, y_test),
                     batch_size=10)

In [0]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [0]:
plot_history(history)

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier

def create_model():
    model = Sequential()
    model.add(Dense(10, input_dim=input_dim, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)

In [0]:
history = model.fit(X_train, y_train,
                     epochs=100,
                     verbose=False,
                     validation_data=(X_test, y_test),
                     batch_size=10)

In [0]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(model).fit(X_train.toarray(), y_train,
                     epochs=100,
                     verbose=False,
                     validation_data=(X_test.toarray(), y_test),
                     batch_size=10)
eli5.show_weights(perm)

In [0]:
eli5.show_weights(perm, feature_names=CVectorizer.get_feature_names())

In [0]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=8213)
tokenizer.fit_on_texts(X)

X_tok = tokenizer.texts_to_sequences(X)

X_train, X_test, y_train, y_test = train_test_split(X_tok, labelled_y, test_size = 0.2, random_state = 42)
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

print(X[2])


In [0]:
from keras.preprocessing.sequence import pad_sequences
maxlen =70

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [0]:
from keras.models import Sequential
from keras import layers
import eli5
from eli5.sklearn import PermutationImportance

def create_model():
    maxlen = 70
    embedding_dim = 50
    
    model = Sequential()
    model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
    model.add(layers.Flatten())
    model.add(layers.Dense(10, activation='relu'))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)


In [0]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(model).fit(X_train, y_train,
                     epochs=100,
                     verbose=False,
                     validation_data=(X_test, y_test),
                     batch_size=10)
eli5.show_weights(perm)

In [0]:
history = model.fit(X_train, y_train,
                    epochs=20,
                    verbose=False,
                    validation_data=(X_test, y_test),
                    batch_size=10)
loss, accuracy = model.score(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.score(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_history(history)

In [0]:
from keras.models import Sequential
from keras import layers

embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', 'precision', 'recall', 'f1-score'])
model.summary()

In [0]:
history = model.fit(X_train, y_train,
                    epochs=100,
                    verbose=False,
                    validation_data=(X_test, y_test),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
print("Testing Precision: {:.4f}".format(precision))
print("Testing Recall: {:.4f}".format(recall))
print("Testing F1Score: {:.4f}".format(f1score))
plot_history(history)

In [0]:
from gensim.models import Word2Vec

model = Word2Vec(, size=100, window=5, min_count=5, workers=16, sg=0, negative=5)